Les données de temperature et d'humidité sont prise de https://climat.meteo.gc.ca/index_f.html

In [1]:
import json
import pandas as pd
from scipy import interpolate
import numpy as np

In [2]:
temps_data = pd.read_csv("fr_climat_horaires_QC_702S006_07-2022_P1H.csv",decimal=',')
temps_data = temps_data[["Jour", "Heure (HNL)","Temp (°C)","Hum. rel (%)","Hauteur de précip. (mm)"]]
temps_data["Heure (HNL)"] = (temps_data["Heure (HNL)"].str.replace(":00","")).astype(int)
temps_data = temps_data[(temps_data["Jour"]>=7) & (temps_data["Jour"]<=13)]
# temps_data

In [3]:
temp_values = [[],[],[]] 
hum_values = [[],[],[]]
water_values = [[],[],[]]
pH_values = [[],[],[]]
co2_values = [[],[],[]]
sun_values = [[],[],[]]
power1 = [[],[],[]]
power2 = [[],[],[]]
power3 = [[],[],[]]
times = []

time_len = [1000,24*5,7*24]
times.append(list(np.linspace(0, time_len[0], num=time_len[0])))
times.append(list(np.linspace(0, 24, num=time_len[1])))
times.append(list(np.linspace(0, 7, num=time_len[2])))

rng = np.random.default_rng(seed=2022)

# for week
ratio2 = 2.5
f1 = interpolate.interp1d(times[2], temps_data["Hum. rel (%)"], kind='cubic')
f2 = interpolate.interp1d(times[2], temps_data["Temp (°C)"], kind='cubic')
mod_time_2 = list(np.linspace(0, 7, num=int(time_len[2]*ratio2)))

hum_tmp = f1(mod_time_2)
hum_values[2] = list(hum_tmp)

temp_tmp = 8+f2(mod_time_2)
temp_values[2] = list(temp_tmp)

w_noise = rng.integers(low=-4, high=5, size=int(time_len[2]*ratio2))
w_noise[w_noise < 0] = 0
water_tmp = np.cumsum((100-hum_tmp)*w_noise*np.exp((1+temp_tmp-np.mean(temp_tmp))/2))/4
water_values[2] = list(water_tmp)

ph_noise = rng.integers(low=-1, high=2, size=int(time_len[2]*ratio2))
ph_noise[ph_noise < 0] = 0

pH_tmp = 6.6 -np.cumsum(np.log(1+water_tmp*100)*0.00008) + np.cumsum(ph_noise*0.003)
pH_tmp = np.mean(pH_tmp)+((pH_tmp-np.mean(pH_tmp))/10)
pH_values[2] = list(pH_tmp)

sun_ratios = [0.92,0.95,0.85,1.0,0.6]
mu,sigma = 6,1.5
sun_tmp = 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (np.array(times[1][:-int(len(times[1])/2)]) - mu)**2 / (2 * sigma**2))
sun_tmp = sun_tmp * 3500
sun_values[2] = list(sun_tmp*0.92)
for r in sun_ratios :
    sun_values[2] = sun_values[2] + list(sun_tmp*r)

co2_tmp = 1400 - pH_tmp*hum_tmp/8*np.log(temp_tmp) + ph_noise*5
co2_tmp = np.mean(co2_tmp)+((co2_tmp-np.mean(co2_tmp))/10)
co2_values[2] = list(co2_tmp)

# for day
ratio1 = 2.75
f1 = interpolate.interp1d(times[2][-25:], temps_data["Hum. rel (%)"][-49:-24], kind='cubic')
f2 = interpolate.interp1d(times[2][-25:], temps_data["Temp (°C)"][-49:-24], kind='cubic')
tmp_x = np.linspace(6,7, num=len(water_values[2][-int(49*ratio2):-int(18*ratio2)]))
f3 = interpolate.interp1d(tmp_x, water_values[2][-int(49*ratio2):-int(18*ratio2)], kind='linear')
f4 = interpolate.interp1d(tmp_x, co2_values[2][-int(49*ratio2):-int(18*ratio2)], kind='linear')
# print(times[2][-25:],new_x)
new_x = list(np.linspace(6,7, num=int(time_len[1]*ratio1)))
times[1] = list(np.linspace(0, 24, num=int(time_len[1]*ratio1)))
hum_values[1] = list(f1(new_x))
temp_values[1] = list(8+f2(new_x))

water_tmp = f3(new_x)
water_values[1] = list(water_tmp-np.min(water_tmp))

ph_noise = rng.integers(low=-1, high=2, size=int(time_len[1]*ratio1))
ph_noise[ph_noise < 0] = 0
pH_tmp = 6.6 -np.cumsum(np.log(1+water_tmp*100)*0.00008) + np.cumsum(ph_noise*0.0035)
pH_tmp = np.mean(pH_tmp)+((pH_tmp-np.mean(pH_tmp))/10)
pH_values[1] = list(pH_tmp)

mu,sigma = 12,3
sun_tmp = 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (np.array(times[1]) - mu)**2 / (2 * sigma**2))
sun_tmp = sun_tmp * 7000
sun_values[1] = list(sun_tmp)

co2_tmp = f4(new_x)
co2_values[1] = list(co2_tmp)

### for real time
tmp = list(np.linspace(0,time_len[0], num=int(time_len[1]*ratio1)))
f1 = interpolate.interp1d(tmp, hum_values[1], kind='linear')
f2 = interpolate.interp1d(tmp, temp_values[1], kind='linear')
f3 = interpolate.interp1d(tmp, water_values[1], kind='linear')
f4 = interpolate.interp1d(tmp, sun_values[1], kind='linear')
f5 = interpolate.interp1d(tmp, co2_values[1], kind='linear')

times[0] = list(np.linspace(0,time_len[0], num=int(time_len[0])))
noise = rng.integers(low=1, high=50, size=time_len[0])

hum_tmp = f1(times[0])
hum_tmp = np.mean(hum_tmp)+((hum_tmp-np.mean(hum_tmp))/10)
hum_tmp = hum_tmp+(noise/100)
hum_values[0] = list(hum_tmp)

temp_tmp = f2(times[0])
temp_tmp = np.mean(temp_tmp)+((temp_tmp-np.mean(temp_tmp))/5)
temp_tmp = temp_tmp+(noise/1000)
temp_values[0] = list(temp_tmp)

water_tmp = f3(times[0])
water_tmp = (water_tmp - np.min(water_tmp))/2 +(noise/10)
water_values[0] = list(water_tmp)

ph_noise = rng.integers(low=-1, high=2, size=time_len[0])
ph_noise[ph_noise < 0] = 0
pH_tmp = 6.6 -np.cumsum(np.log(1+water_tmp*100)*0.00008) + np.cumsum(ph_noise*0.0023)
pH_tmp = np.mean(pH_tmp)+((pH_tmp-np.mean(pH_tmp))/10)
pH_values[0] = list(pH_tmp)

sun_tmp = f4(times[0])
sun_tmp = sun_tmp+(noise/2)
sun_values[0] = list(sun_tmp)

co2_tmp = f5(times[0])
co2_tmp = co2_tmp+(noise/50)
co2_values[0] = list(co2_tmp)

times[2] = mod_time_2

#For power estimate
cost = 0.05

powerdraw1 = [cost*int((i%4/3)) for i in range(330)]
powerdraw2 = [cost*int((i%6/5)) for i in range(330)]
powerdraw3 = [3*cost*int((i%10)/9) for i in range(330)]

power1[1]=list(100 - np.cumsum(powerdraw1))
power2[1]=list(100 - np.cumsum(powerdraw2))
power3[1]=list(100 - np.cumsum(powerdraw3))


Data = {"times":times,"temp_values":temp_values,"hum_values":hum_values,"water_values":water_values,"pH_values":pH_values,"CO2_values":co2_values,"Sun_values":sun_values,"power1":power1,"power2":power2,"power3":power3}
with open('data.json', 'w') as fp:
    json.dump(Data, fp)